In [8]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey
import requests
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "liar"

In [9]:
# autoryzacja

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [10]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'send me any question in english, and I will try to answer it in max 150 tokens', 'hint1': "please send your question in 'question' field to /task/ endpoint (simple form, not JSON)", 'hint2': "sometimes I don't tell the truth", 'hint3': "Send to /answer/ info if I'm telling the truth. Just value: YES/NO"}


In [12]:
question = "What is capital of Poland?"
data = {"question": question}
url = f"https://tasks.aidevs.pl/task/{token}"

chat = ChatOpenAI(
    model='gpt-4'
)

try:
    response = requests.post(url, data=data)
    response.raise_for_status()  # Sprawdza, czy odpowiedź jest sukcesem (status kod 2xx)
    print(response.json())

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")

guard_prompt = f"Return 'YES' or 'NO' if the answer: '{response.json()['answer']}' is to the question: '{question}'. Answer:"
prompt_template = PromptTemplate.from_template(guard_prompt) # a tutaj było to chyba koniecznie bo bez tego przy tworzeniu obiektu LLMChain był błąd

chain = LLMChain(llm=chat, prompt=prompt_template)

# Wywołaj chain z treścią odpowiedzi ?
result = chain.run(response=response.json()['answer'])

if result == 'YES':
    print("Answer on topic")
else:
    print("WTF is the answer")

{'code': 0, 'msg': 'This is my answer', 'answer': 'The iconic "Welcome to Jurassic Park" scene was filmed on the Hawaiian island of Kauai.'}
WTF is the answer


In [13]:
# wysłanie odpowiedzi

response = send_answer(token, result)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
